This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [79]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'NuwxyuPGQX9WzsfJ-YPm'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [80]:
# First, import the relevant modules
import requests
import collections
import json

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [272]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?start_date=2020-03-16&end_date=2020-03-16&api_key='+API_KEY)
r.json()

{'dataset': {'id': 10095370,
  'dataset_code': 'AFX_X',
  'database_code': 'FSE',
  'name': 'Carl Zeiss Meditec (AFX_X)',
  'description': 'Stock Prices for Carl Zeiss Meditec (2020-03-02) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704',
  'refreshed_at': '2020-03-16T22:16:31.077Z',
  'newest_available_date': '2020-03-16',
  'oldest_available_date': '2000-06-07',
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'frequency': 'daily',
  'type': 'Time Series',
  'premium': False,
  'limit': None,
  'transform': None,
  'column_index': None,
  'start_date': '2020-03-16',
  'end_date': '2020-03-16',
  'data': [['2020-03-16',
    75.0,
    76.9,
    71.45,
    76.15,
    None,
    422533.0,
    31664759.85,
    None,
    None,
    None]],
  'collapse': None,
  'order': None,
  'database_id': 6129}}

Comparing today's information to the querying google shows that this price information is denominated in euros not USD
as I had originally assumed. It is the Frankfurt Stock Exchange after all.

In [252]:
r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-01-01&end_date=2017-12-31&api_key='+API_KEY)
datajson = dict(r.json())['dataset']

While I would normally just put these dicts into a numpy array or pandas DataFrame the exercise discourages this.
As such, I think the easiest way of handling the data, because it exists in a list of lists, is to simply iterate
over its elements and extract the requested values. The order of the elements of each list are:

In [255]:
datajson['column_names']

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Change',
 'Traded Volume',
 'Turnover',
 'Last Price of the Day',
 'Daily Traded Units',
 'Daily Turnover']

The data is stored in the key 'data'; iterate over the data and use different built-in functions to
compute the requested quantities. Some values are None type so the conditional statements are either
precautionary or required.

In [231]:
highest_open = 0
largest_intraday_change = 0 
largest_interday_change = 0
lowest_open = 10000000 
traded_volume = []
for i, d in enumerate(datajson['data']):
    if d[1] is not None:
        highest_open = max(d[1], highest_open)
        lowest_open = min(d[1], lowest_open)
        if d[2] is not None:
            largest_intraday_change = max(d[2]-d[1], largest_intraday_change)
    if d[4] is not None:
        if i==0:     
            previous_day_close = d[4]  
        else:
            largest_interday_change =  max(d[4]-previous_day_close, largest_interday_change)
            previous_day_close=d[4]
    traded_volume+=[d[6]]

Calculate what the highest and lowest opening prices were for the stock in this period.

In [290]:
print('The highest opening price was %.2f €.' % highest_open)

The highest opening price was 53.11 €.


In [291]:
print('The lowest opening price was %.2f €.' % lowest_open)

The lowest opening price was 34.00 €.


What was the largest change in any one day (based on High and Low price)?

In [292]:
print('The largest intraday change was %.2f €.' % largest_intraday_change)

The largest intraday change was 2.66 €.


What was the largest change between any two days (based on Closing Price)?

In [293]:
print('The largest interday change was %.2f €.' % largest_interday_change)

The largest interday change was 2.56 €.


What was the average daily trading volume during this year?

In [294]:
print('The lowest opening price was %.2f €.' % lowest_open)

The lowest opening price was 34.00 €.


(Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [298]:
def my_median(listlike):
    if len(traded_volume) % 2:
        median=traded_volume[len(traded_volume)//2 + 1]
    else:
        # Take the average if there are an even number of elements
        median=(traded_volume[len(traded_volume)//2]+traded_volume[len(traded_volume)//2 + 1]) / 2.0
    return median

In [299]:
print('The median was %d €.' % my_median(traded_volume))

The median was 76600 €.
